In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("./dumpdata.xlsx")
df.head()

,RoomCode,UserId,ClosedBy,ClosingComment,ChatStartTime,ChatEndTime,AgentAssignmentTimestamp,BotFirstResponseTime,AgentFirstResponseTime,CsatScore
0,8528073,e2ff5f48-5f2b-4bb6-894b-fad28843fe0b,System,Closing inactive room,2021-04-02 10:39:34,2021-04-02 11:40:20,-,00:00:00,-,0
1,8674865,362f984a-1487-4fd8-85b2-1e2a94442f9b,System,Closing now,26-02-2021 12:51:37,26-02-2021 12:55:27,-,00:00:00,-,0
2,8543522,4ae0951a-4ba4-4e2f-863b-eb1fab965933,System,Closing inactive room,2021-06-02 12:17:00,2021-06-02 13:17:07,-,00:00:00,-,0
3,8581336,b675563d-8906-4b9d-9368-fda6f21b16d5,System,Closing now,2021-11-02 19:24:24,2021-11-02 19:25:03,-,00:00:01,-,0
4,8613422,b254a88e-4b76-4667-8352-f1583d71da1a,Nisha,Unresponsive Customer,16-02-2021 14:37:17,16-02-2021 14:59:54,16-02-2021 14:39:56,00:00:00,00:00:04,3


## Data Preprocessing

In [3]:
df['AgentFirstResponseTime'] = df['AgentFirstResponseTime'].replace('-', pd.to_datetime(0, unit='s').time())
ChatStartTime = pd.to_datetime(df['ChatStartTime'], errors='coerce')
ChatEndTime = pd.to_datetime(df['ChatEndTime'], errors='coerce')

In [4]:
# Swapping day value with month value for single digit dates to make data consistent

def standardize_date(dt):
    if ((1<=dt.day) & (dt.day<=9)):
        month = dt.day
        dt = dt.replace(day=dt.month)
        dt = dt.replace(month=month)
    return dt

In [5]:
df['ChatStartTime'] = ChatStartTime.apply(standardize_date)
df['ChatEndTime'] = ChatEndTime.apply(standardize_date)
df.head()

,RoomCode,UserId,ClosedBy,ClosingComment,ChatStartTime,ChatEndTime,AgentAssignmentTimestamp,BotFirstResponseTime,AgentFirstResponseTime,CsatScore
0,8528073,e2ff5f48-5f2b-4bb6-894b-fad28843fe0b,System,Closing inactive room,2021-02-04 10:39:34,2021-02-04 11:40:20,-,00:00:00,00:00:00,0
1,8674865,362f984a-1487-4fd8-85b2-1e2a94442f9b,System,Closing now,2021-02-26 12:51:37,2021-02-26 12:55:27,-,00:00:00,00:00:00,0
2,8543522,4ae0951a-4ba4-4e2f-863b-eb1fab965933,System,Closing inactive room,2021-02-06 12:17:00,2021-02-06 13:17:07,-,00:00:00,00:00:00,0
3,8581336,b675563d-8906-4b9d-9368-fda6f21b16d5,System,Closing now,2021-02-11 19:24:24,2021-02-11 19:25:03,-,00:00:01,00:00:00,0
4,8613422,b254a88e-4b76-4667-8352-f1583d71da1a,Nisha,Unresponsive Customer,2021-02-16 14:37:17,2021-02-16 14:59:54,16-02-2021 14:39:56,00:00:00,00:00:04,3


## Dividing Data into weeks

In [6]:
# Index of weeks list corresponds to data in that particular week
days =  df['ChatStartTime'].dt.day

weeks = [
    df[(1 <=days) &  (days <= 7)], df[(8 <=days) &  (days <= 14)], 
    df[(15 <=days) &  (days <= 21)], df[(22 <=days) &  (days <= 28)] 
]

# Chart Report

In [7]:
# Createing chart report dataframe

chartReport = pd.DataFrame(columns=['Week', 'Incoming Chats', 'Unique Users', 'Closed By Bot', 'Bot Deflection %', 'Closed By Agents']).reset_index(drop=True)
chartReport['Week'] = ["01 Feb - 07 Feb", "08 Feb - 14 Feb", "15 Feb - 21 Feb", "22 Feb - 28 Feb"]
chartReport

,Week,Incoming Chats,Unique Users,Closed By Bot,Bot Deflection %,Closed By Agents
0,01 Feb - 07 Feb,NaN,NaN,NaN,NaN,NaN
1,08 Feb - 14 Feb,NaN,NaN,NaN,NaN,NaN
2,15 Feb - 21 Feb,NaN,NaN,NaN,NaN,NaN
3,22 Feb - 28 Feb,NaN,NaN,NaN,NaN,NaN


In [8]:
# Applying filter on each column according to requirenemt.

def applyChartFilter(week):
    incomingChats = len(week["RoomCode"])
    userid =  len(pd.unique(week["UserId"]))
    closedByBot = len(week[week['ClosedBy'].isin(["System"])])
    botDeflection = round(closedByBot/incomingChats,4)*100
    closedByAgents = len(week.loc[week['ClosedBy'] != 'System']) 
    return [incomingChats, userid, closedByAgents, botDeflection, closedByAgents]

In [9]:
# Looping through chartReport dataframe and populating it with each week data individually

for i, week in enumerate(weeks, start=0):
    weekData = applyChartFilter(week)
    for j in range(1,6):
        chartReport.iloc[i,j] = weekData[j-1]
chartReport

,Week,Incoming Chats,Unique Users,Closed By Bot,Bot Deflection %,Closed By Agents
0,01 Feb - 07 Feb,2763,2642,975,64.71,975
1,08 Feb - 14 Feb,2620,2513,952,63.66,952
2,15 Feb - 21 Feb,2286,2188,922,59.67,922
3,22 Feb - 28 Feb,2329,2230,883,62.09,883


# Agent Report

In [10]:
# Createing agent report dataframe

agentReport = pd.DataFrame(columns=['Week', 'Agent Name', 'Chats Resolved', 'Avg Agent First Response Time (seconds)', 'Avg Agent Chat Resolution Time (seconds)', 'Average Agent CSAT Score', 'Business Hours CSAT', 'Outside Business Hours CSAT']).reset_index(drop=True)
agentReport['Week'] = ["01 Feb - 07 Feb", "08 Feb - 14 Feb", "15 Feb - 21 Feb", "22 Feb - 28 Feb"]
agentReport

,Week,Agent Name,Chats Resolved,Avg Agent First Response Time (seconds),Avg Agent Chat Resolution Time (seconds),Average Agent CSAT Score,Business Hours CSAT,Outside Business Hours CSAT
0,01 Feb - 07 Feb,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,08 Feb - 14 Feb,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15 Feb - 21 Feb,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,22 Feb - 28 Feb,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# First populating only agent names column by storing names for 

for i, week in enumerate(weeks, start=0):
    AgentNames = pd.unique(week.loc[week['ClosedBy'] != 'System']['ClosedBy'])
    agentReport.iat[i,1] = AgentNames



In [12]:
# Now applying explode method on agent name column to map each name from list to their corresponding week value

agentReport = agentReport.explode('Agent Name')
agentReport

,Week,Agent Name,Chats Resolved,Avg Agent First Response Time (seconds),Avg Agent Chat Resolution Time (seconds),Average Agent CSAT Score,Business Hours CSAT,Outside Business Hours CSAT
0,01 Feb - 07 Feb,Kabir,NaN,NaN,NaN,NaN,NaN,NaN
0,01 Feb - 07 Feb,Samy,NaN,NaN,NaN,NaN,NaN,NaN
0,01 Feb - 07 Feb,Nisha,NaN,NaN,NaN,NaN,NaN,NaN
0,01 Feb - 07 Feb,Isha,NaN,NaN,NaN,NaN,NaN,NaN
0,01 Feb - 07 Feb,Rozeta,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
3,22 Feb - 28 Feb,Rebecca,NaN,NaN,NaN,NaN,NaN,NaN
3,22 Feb - 28 Feb,Daanish,NaN,NaN,NaN,NaN,NaN,NaN
3,22 Feb - 28 Feb,Zia,NaN,NaN,NaN,NaN,NaN,NaN
3,22 Feb - 28 Feb,Pritam,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Defining function to calculate csat score with dataframe which is selected based on individual name followed by that particular week

def calculate_csat_score(df):
    positive_csat_counts = len(df[df['CsatScore'].isin([4, 5])])
    negative_csat_counts = len(df[df['CsatScore'].isin([1, 2, 3])])
    if positive_csat_counts != 0:
        avg_csat = positive_csat_counts / (positive_csat_counts + negative_csat_counts)
    else:
        avg_csat = 0
    return round(avg_csat, 2)

In [14]:
# Applying filter on each column according to requirenemt.

def agentReportFilter(row):
    data = weeks[0] if row.iloc[0] == "01 Feb - 07 Feb" else weeks[1] if row.iloc[0] == "08 Feb - 14 Feb" else weeks[2] if row.iloc[0] == "15 Feb - 21 Feb" else weeks[3]
    data = data.loc[data['ClosedBy'] == row.iloc[1]]
    row.iloc[2] = len(data) # Chats Resolved
    sec_avg = round(data['AgentFirstResponseTime'].apply(lambda t: t.hour * 3600 + t.minute * 60 + t.second).mean(), 2) 
    row.iloc[3] = sec_avg  # Avg Agent First Response Time (seconds)
    row.iloc[4] = round(((data['ChatEndTime'] - data['ChatStartTime']).dt.total_seconds()).mean(), 2) # Avg Agent Chat Resolution Time (seconds)
    row.iloc[5] = calculate_csat_score(data) # Average Agent CSAT Score
    start_time = pd.to_datetime('10:00:00').time()
    end_time = pd.to_datetime('17:00:00').time()
    business_hours = data[(data['ChatStartTime'].dt.time >= start_time) & (data['ChatEndTime'].dt.time <= end_time)]
    row.iloc[6] = calculate_csat_score(business_hours) # Business Hours CSAT
    non_business_hours = data[(data['ChatStartTime'].dt.time < start_time) | (data['ChatEndTime'].dt.time > end_time)]
    row.iloc[7] = calculate_csat_score(non_business_hours) # Outside Business Hours CSAT
    return row

In [15]:
# Applying filter function row by row to calculate column values for all individuals seperately based on individual name and week value

agentReport = agentReport.apply(agentReportFilter, axis=1)
agentReport

,Week,Agent Name,Chats Resolved,Avg Agent First Response Time (seconds),Avg Agent Chat Resolution Time (seconds),Average Agent CSAT Score,Business Hours CSAT,Outside Business Hours CSAT
0,01 Feb - 07 Feb,Kabir,8,28.50,869.50,0.25,0.00,0.5
0,01 Feb - 07 Feb,Samy,19,21.16,739.00,1.00,1.00,0.0
0,01 Feb - 07 Feb,Nisha,11,25.82,1302.45,0.29,0.00,1.0
0,01 Feb - 07 Feb,Isha,28,5.25,745.32,0.75,0.82,0.0
0,01 Feb - 07 Feb,Rozeta,8,8.12,1594.50,0.50,0.00,0.5
...,...,...,...,...,...,...,...,...
3,22 Feb - 28 Feb,Rebecca,5,42.60,1341.40,0.25,0.00,0.5
3,22 Feb - 28 Feb,Daanish,9,9.44,604.56,0.00,0.00,0.0
3,22 Feb - 28 Feb,Zia,4,12.25,938.75,1.00,1.00,1.0
3,22 Feb - 28 Feb,Pritam,4,7.50,902.50,0.50,0.50,0.0


In [16]:
# Writing chartreport and agent report dataframe into excel and saving it

with pd.ExcelWriter('output_file.xlsx') as writer:
    # Write each DataFrame to a different sheet
    chartReport.to_excel(writer, sheet_name='Chart Report', index=False)
    agentReport.to_excel(writer, sheet_name='Agent Report', index=False)